In [58]:
import urllib
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import pickle
from collections import Counter
from collections import namedtuple
import re
import xlwings as xw
import pandas as pd
import numpy as np

In [59]:
path_to_names = "D:/"
path_to_patronymics = "D:/"

In [60]:
with open(path_to_names+"NamesFromGufo.pickle","rb") as f:
    tmpNames=pickle.load(f)
with open(path_to_patronymics+"PatronymicsFromGufo.pickle","rb") as f:
    tmpPatronymics=pickle.load(f)

In [61]:
all_names=[]
all_patronymics=[]

In [62]:
for i in tmpNames:
    all_names.append(i)
for i in tmpPatronymics:
    all_patronymics.append(i)

In [63]:
def distance(s, t):
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]

In [64]:
def checkSurnames(s):
    pattern = '\w*(ов|ова|ев|ёв|ева|ёва|ив|ин|ина|ын|их|ых|ский|цкий|ая|ко)$'
    if (re.match(pattern,s)): return True
    else: return False

In [65]:
def checkPatronymic(s):
    pattern = '\w*(ович|евич|ич|овна|евна|ична|инична)$'
    if (re.match(pattern,s)):
        return True
    else: return False

In [66]:
genderTuple = ('male', 'female', 'unknown')

In [67]:
def checkSurnamesGender(s):
    patternMale = '\w*(ов|ев|ий|ын|ин)$'
    patternFem ='\w*(ова|ева|ая|ина|ына)$'
    patternUnknown = '\w*(их|ых|ко|ук|юк|ун|ний|ный|чай|ий|а|ич|ов|ук|ик|ски|ка|ски|цки|дзки)$'
    
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    elif (re.match(patternUnknown,s)):
        return genderTuple[2]
    else: return 'No'

In [68]:
def changeGenderSurname(s):
    patternMaleA = '\w*(ов|ев|ин|ын)$'
    patternFemA ='\w*(ова|ева|ина|ына)$'
    patternMaleB = '\w*(ий)$'
    patternFemB = '\w*(ая)$'
    
    if(re.match(patternMaleA, s)):
        return s + 'a'
    elif (re.match(patternMaleB, s)):       
        s += ' '
        return s.replace('ий ', 'ая')
    elif (re.match(patternFemA, s)):
        return s[0: len(s)-1]
    elif (re.match(patternFemB, s)): 
        s += ' '
        return s.replace('ая ', 'ий')
    else: return s

In [69]:
def checkPatronymicGender(s):
    patternMale = '\w*(ович|евич|ич)$'
    patternFem ='\w*(овна|евна|ична|инична)$'
   
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    else: return 'No'    

In [70]:
def changeGenderPatronymic(s):
    patternMaleA = '\w*(ович|евич)$'
    patternFemA ='\w*(овна|евна)$'
    patternMaleB = '\w*(ич)$'
    patternFemB = '\w*(ична|инична)$'

    if(re.match(patternMaleA, s)):
        s += ' '
        return s.replace('ич ', 'на')
    if(re.match(patternFemA, s)):
        s += ' '
        return s.replace('на ', 'ич')     
    if(re.match(patternMaleB, s)):
        return s + 'на'   
    if(re.match(patternFemB, s)):
        s += ' '
        s = s.replace('инична ', 'ич')
        s = s.replace('ична ', 'ич')
        return s    
    else: return s

In [71]:
def Brute(tmp):
    res = []
    errs=[]
    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        words = t.split(" ")
        for w in words:
            w = w.lower()
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and surname == ""):
                surname = w

        if (name == ""):
            name = "N:'"+w+"'_Not_Found"
            #errs.append("name '" + w + "' in " + t)
        if (patronymic == ""):
            patronymic = "P:'"+w+"'_Not_Found"
            errs.append("patronymic '" + w + "' in " + t)
        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    if len(errs)!=0:
        res.append("----Ошибки----")
        for i in errs:
            res.append(i)
    return res

In [72]:
def NotBruteEnough(tmp):
    res = []

    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        t = t.lower()
        words = t.split(" ")

        for w in words:
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and checkSurnames(w) and surname == ""):
                surname = w

        if (name in words): words.remove(name)
        if (patronymic in words): words.remove(patronymic)
        if (surname in words): words.remove(surname)

        if (name == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for n in all_names:
                    dist = distance(w, n)
                    if dist < minDist:
                        minDist = dist
                        name = n.title() + " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (patronymic == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = distance(w, p)
                    if dist < minDist:
                        minDist = dist
                        patronymic = p.title() + " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (surname == "")& (len(words) > 0):
            surname = words[0]

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    return res

In [73]:
def detectSurname(w):
    w = w.lower()
    if (checkSurnames(w)):
        return [w, 1.0]
    else: 
        return [w, 0.5] #Ну пока так, фамилии разные бывают. Когда будет справочник, сделаем по-другому 

In [77]:
def detectName(w):
    w = w.lower()
    if (w in all_names):
        return [w, 1.0]
    else:
        name = w
        minDist = len(w)
        for n in all_names:
            dist = distance(w, n)
            if dist < minDist:
                minDist = dist
                name = n
        probability = round(1 - minDist / len(w), 2)
        return [name, probability]

In [78]:
def detectPatronymic(w):
    w = w.lower()
    if (w in all_patronymics):
        return [w, 1.0]
    #elif (checkPatronymic(w)):
    #    return [w.title(), 0.90] #да, от балды, потом исправим
    else:
        patronymic = w
        minDist = len(w)
        for p in all_patronymics:
            dist = distance(w, p)
            if dist < minDist:
                minDist = dist
                patronymic = p
        probability = round(1 - minDist / len(w), 2)
        return [patronymic, probability]

In [129]:
def FIO(words):
    surname=detectSurname(words[0])
    name=detectName(words[1])
    patronymic=detectPatronymic(words[2])
    return [surname, name, patronymic]

In [130]:
def IOF(words):
    surname=detectSurname(words[2])
    name=detectName(words[0])
    patronymic=detectPatronymic(words[1])
    return [surname, name, patronymic]

In [131]:
def IFO(words):
    surname=detectSurname(words[1])
    name=detectName(words[0])
    patronymic=detectPatronymic(words[2])
    return [surname, name, patronymic]

In [132]:
def ElseOptions(words): #Настолько кривой, насколько возможно
    surname = ["",0] 
    name= ["",0] 
    patronymic = ["",0] 
    
    for w in words:
        w.strip()
        
        if (name[0] == ""):
            if (w in all_names):
                name = [w,1.0]
        if (patronymic[0] == ""):
            if (w in all_patronymics):
                patronymic = [w,1.0]
        if (name[0] != w and patronymic[0] != w and checkSurnames(w) and surname[0] == ""):
            surname = [w,1.0]

    if (name[0] in words): words.remove(name[0])
    if (patronymic[0] in words): words.remove(patronymic[0])
    if (surname[0] in words): words.remove(surname[0])

    if (name[0] == "") & (len(words) > 0):
        foundWord = ""
        for w in words:
            minDist = len(w)
            for n in all_names:
                dist = distance(w, n)
                if dist <= minDist:
                    minDist = dist
                    name[0] = n
                    foundWord = w
            name[1] = round(1 - minDist / len(w), 2)
        words.remove(foundWord)
    
    if (patronymic[0] == "") & (len(words) > 0):
        foundWord = ""
        for w in words:
            minDist = len(w)
            for p in all_patronymics:
                dist = distance(w, p)
                if dist <= minDist:
                    minDist = dist
                    patronymic[0] = p
                    foundWord = w

            patronymic[1] = round(1 - minDist / len(w), 2)
        words.remove(foundWord)
    
    if (surname[0] == "")& (len(words) > 0):
        surname = [words[0], 0.5]
    return [surname, name, patronymic]

In [133]:
def FI(words):
    surname=detectSurname(words[0])
    name=detectName(words[1])
    return [surname, name]

In [134]:
def IF(words):
    surname=detectSurname(words[1])
    name=detectName(words[0])
    return [surname, name]

In [147]:
def HypothesisSearch(tmp):#не доделан
    res = []
    
    hypo3 = {FIO: 0.40, IOF: 0.35, IFO: 0.25, ElseOptions: 0.05}
    #hypo2 = {FI: 0.5, IF: 0.5}
    #hypo = {FIO: 0.60, IOF: 0.40}
    for t in tmp:
        t = t.lower()
        words = t.split(" ")
        for w in words: w.strip()
        
        if (len(words)==3):
            method = lambda x:0
            max = 0
            for h in hypo3:
                if (hypo3[h]>max):
                    max = hypo3[h]
                    method = h
            res.append(method(words))
        #elif (len(words)==2):
                    
        else: 
            res.append(t)
    return res
print(HypothesisSearch(["Бучнева Анна Всеволодовна"]))

[[['бучнева', 1.0], ['анна', 1.0], ['всеволодовна', 1.0]]]


In [109]:
with open("D:/test_changed.pickle","rb") as f:
    tmp=pickle.load(f)

In [110]:
testing_res=NotBruteEnough(tmp)
for fio in testing_res:
    print(fio)

In [ ]:
errs=0
for i in testing_res:
    if (i.find("%")!=-1):
        print (i)
        errs+=1
print("Всего не уверены в: " + str(errs))